In [4]:
import pandas as pd

data = pd.read_csv("books.csv")


# Since Publisher and Author has NaN values, we will replace them with "Unknown"
data["Publisher"] = data["Publisher"].fillna("Unknown")
data["Author"] = data["Author"].fillna("Unknown")

data_books = data.to_dict("records")

len(data)

211

In [5]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-MiniLM-L6-v2")  # Model to create embeddings

# Create the vector database client
qdrant = QdrantClient(":memory:")  # Create in-memory Qdrant instance

c:\Users\LauraGalvisBarona\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Create collection to store wines
qdrant.create_collection(
    collection_name="books_collection",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [7]:
# Vectorize and upload books
qdrant.upload_points(
    collection_name="books_collection",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Genre"]).tolist(),
            payload=doc,
        )
        for idx, doc in enumerate(data_books)
    ],
)

In [8]:
user_prompt = "Suggest me an interesting history book"

In [9]:
# Search time for awesome books!

hits = qdrant.search(
    collection_name="books_collection",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=5,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{'Title': 'Mossad', 'Author': 'Baz-Zohar, Michael', 'Genre': 'history', 'Height': 236, 'Publisher': 'Unknown'} score: 0.5355729212558695
{'Title': 'History of England, Foundation', 'Author': 'Ackroyd, Peter', 'Genre': 'history', 'Height': 197, 'Publisher': 'Unknown'} score: 0.5355729212558695
{'Title': 'City of Djinns', 'Author': 'Dalrymple, William', 'Genre': 'history', 'Height': 198, 'Publisher': 'Unknown'} score: 0.5355729212558695
{'Title': 'Empire of the Mughal - Ruler of the World', 'Author': 'Rutherford, Alex', 'Genre': 'history', 'Height': 180, 'Publisher': 'Unknown'} score: 0.5355729212558695
{'Title': "Empire of the Mughal - The Serpent's Tooth", 'Author': 'Rutherford, Alex', 'Genre': 'history', 'Height': 180, 'Publisher': 'Unknown'} score: 0.5355729212558695


C:\Users\LauraGalvisBarona\AppData\Local\Temp\ipykernel_5004\3555496826.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [10]:
# Define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [12]:
import os
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()  # Load environment variables


# Configuring AzureChatOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_service = os.getenv("OPENAI_SERVICE")


llm = AzureChatOpenAI(

    azure_deployment="gpt-4o-mini",

    api_key=openai_api_key,

    azure_endpoint=f"https://{openai_service}.openai.azure.com/",

    api_version="2024-08-01-preview",

)


# Creating chat message

messages = [

    {

        "role": "system",

        "content": "You are chatbot, a book specialist. Your top priority is to help guide users into selecting amazing book according their favorite genre.",

    },

    {"role": "user", "content": user_prompt},

    {"role": "assistant", "content": str(data_books)},
]


# Obtener la respuesta del modelo

completion = llm.invoke(model="gpt-4o-mini", input=messages)


# Imprimir la respuesta

print(completion.content)

Here are some interesting history books you might enjoy:

1. **"Orientalism" by Edward Said** - This groundbreaking work examines Western perceptions of the East and how these perspectives shaped historical and cultural interactions.

2. **"Age of Wrath: A History of the First Crusade" by Abraham Eraly** - A detailed narrative that explores the causes, events, and consequences of one of the significant events in history.

3. **"Last Mughal, The" by William Dalrymple** - A captivating account of the life of the last Mughal emperor, Bahadur Shah II, and the fall of the Mughal Empire in India.

4. **"Freedom at Midnight" by Dominique Lapierre and Larry Collins** - A riveting account of India's struggle for independence, focusing on the final days of British rule and the partition of India.

5. **"All the President's Men" by Bob Woodward and Carl Bernstein** - A classic in political history that details the investigative journalism that uncovered the Watergate scandal.

6. **"Clash of Civi